In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import rpy2.robjects as ro
import rpy2.robjects.pandas2ri as pandas2ri
from rpy2.robjects.packages import importr

In [ ]:
# Importare DataFrame
df = pd.read_excel(r"/Users/vigji/Downloads/Dataframe_Originale.xlsx")

# Stampare DataFrame Originale
print("Initial DataFrame:")
print(df)

In [ ]:
# Tenere solo le colonne utili per le analisi
selected_columns = ['Condition','Training','Duration_P1','Duration_P2','Duration_P3','Order_P1','Order_Training','Order_P2','FL_367_DO','FL_373_DO','AccQ1_P1','AccQ2_P1','AccQ3_P1','AccQ4_P1','AccQ1_P2','AccQ2_P2','AccQ3_P2','AccQ4_P2','AccQ1_P3','AccQ2_P3','AccQ3_P3','AccQ4_P3','AccQ5_P3','AccQ6_P3','AccQ7_P3','AccQ8_P3']

# Creare un nuovo DataFrame con solo le colonne selezionate
simplified_data_pp = df[selected_columns]

# Creare una copia del DataFrame
simplified_data = simplified_data_pp.copy()

# Aggiungere una colonna Subject_ID basata sull'indice
simplified_data['Subject_ID'] = simplified_data_pp.index + 1  # Aggiunge un ID unico ai soggetti

# Rinominare le colonne FL_367_DO e FL_373_DO
simplified_data.rename(columns={'FL_367_DO': 'Order_P3a', 'FL_373_DO': 'Order_P3b'}, inplace=True)

#Settare Ordine di presentazione come variabile categoriale
simplified_data['Order_P1'] = simplified_data['Order_P1'].astype('category').cat.codes
simplified_data['Order_Training'] = simplified_data['Order_Training'].astype('category').cat.codes
simplified_data['Order_P2'] = simplified_data['Order_P2'].astype('category').cat.codes
simplified_data['Order_P3a'] = simplified_data['Order_P3a'].astype('category').cat.codes
simplified_data['Order_P3b'] = simplified_data['Order_P3b'].astype('category').cat.codes

# Stampare il nuovo DataFrame
print("DataFrame after selecting specific columns:")
print(simplified_data)

In [ ]:
# Rimuovere righe con celle NaN
data = simplified_data.dropna()
# Stampare nuovo DataFrame
print("DataFrame after removing rows with NaN value in any column:")
print(data)

In [ ]:
# Trovare media e ds nei tempi di realizzazione
mean_times = data[['Duration_P1', 'Duration_P2', 'Duration_P3']].mean()
std_times = data[['Duration_P1', 'Duration_P2', 'Duration_P3']].std()

print("\nMean times:")
print(mean_times)

print("\nStandard Deviation times:")
print(std_times)

In [ ]:
# Identificare i soggetti fuori dal range 2 SD dalla media Duration
outliers = ((data[['Duration_P1', 'Duration_P2', 'Duration_P3']] > (mean_times + 2 * std_times)) | 
            (data[['Duration_P1', 'Duration_P2', 'Duration_P3']] < (mean_times - 2 * std_times))).all(axis=1)

# Rimuovere i soggetti outlier
filtered_data = data[~outliers].copy()

# Stampare nuovo DataFrame
print("\nFiltered DataFrame (outliers removed):")
print(filtered_data)

In [ ]:
# Trasformare in formato lungo
data_long = pd.melt(
    filtered_data,
    id_vars=['Subject_ID', 'Condition', 'Training'],
    value_vars=['AccQ1_P1', 'AccQ2_P1', 'AccQ3_P1', 'AccQ4_P1',
                'AccQ1_P2', 'AccQ2_P2', 'AccQ3_P2', 'AccQ4_P2',
                'AccQ1_P3', 'AccQ2_P3', 'AccQ3_P3', 'AccQ4_P3',
                'AccQ5_P3', 'AccQ6_P3', 'AccQ7_P3', 'AccQ8_P3'],
    var_name='Phase',
    value_name='Accuracy'
)

#Sistemare formati variabili
data_long['Subject_ID'] = data_long['Subject_ID'].astype('category')
data_long['Condition'] = data_long['Condition'].astype('category')
data_long['Training'] = data_long['Training'].astype('category')
data_long['Phase'] = data_long['Phase'].astype('category')
data_long['Accuracy'] = data_long['Accuracy'].astype(int)

# Verificare il risultato
print(data_long.dtypes)

# Mappare le fasi a etichette (P1: Fase 1, P2: Fase 2, etc.)
phase_mapping = {
    'AccQ1_P1': 'Fase 1', 'AccQ2_P1': 'Fase 1', 'AccQ3_P1': 'Fase 1', 'AccQ4_P1': 'Fase 1',
    'AccQ1_P2': 'Fase 2', 'AccQ2_P2': 'Fase 2', 'AccQ3_P2': 'Fase 2', 'AccQ4_P2': 'Fase 2',
    'AccQ1_P3': 'Fase 3', 'AccQ2_P3': 'Fase 3', 'AccQ3_P3': 'Fase 3', 'AccQ4_P3': 'Fase 3',
    'AccQ5_P3': 'Fase 4', 'AccQ6_P3': 'Fase 4', 'AccQ7_P3': 'Fase 4', 'AccQ8_P3': 'Fase 4'
}

data_long['Phase'] = data_long['Phase'].map(phase_mapping)

# Verificare il risultato
print(data_long)

In [ ]:
# Calcolare l'accuratezza media per Fase, Condizione e Training
mean_accuracy = data_long.groupby(['Phase', 'Condition', 'Training'])['Accuracy'].mean().reset_index()


In [ ]:
# Impostare la dimensione della figura
plt.figure(figsize=(12, 6))

# Creare un grafico a linee
sns.lineplot(data=mean_accuracy,
             x='Phase',
             y='Accuracy',
             hue='Condition', #Impostare Colore in base a 'Condition'
             style='Training', #Impostare Stile in base a 'Training'
             markers=True, dashes=False)

# Aggiungi titoli e etichette
plt.title('Accuratezza Media per Fase, Condizione e Training')
plt.xlabel('Phase')
plt.ylabel('Mean Accuracy')
plt.legend(title='Condition and Training')
plt.grid(True)
plt.tight_layout()

# Mostra il grafico
plt.show()

In [ ]:
# Istogramma e grafico della densità per verifica normalità
plt.figure(figsize=(12, 6))
sns.histplot(data_long['Accuracy'], kde=True)
plt.title('Istogramma Accuracy')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')
plt.show()

# Q-Q plot per la distribuzione normale
plt.figure(figsize=(6, 6))
stats.probplot(data_long['Accuracy'], dist="norm", plot=plt)
plt.title('Q-Q Plot della Accuracy')
plt.show()

#Eseguire il Test di Normalità
# Test di Shapiro-Wilk
stat, p = stats.shapiro(data_long['Accuracy'])
print(f'Shapiro-Wilk Test: Statistic={stat:.3f}, p-value={p:.3f}')
    
# Test di Kolmogorov-Smirnov
stat, p = stats.kstest(data_long['Accuracy'], 'norm', args=(data_long['Accuracy'].mean(), data_long['Accuracy'].std()))
print(f'Kolmogorov-Smirnov Test: Statistic={stat:.3f}, p-value={p:.3f}')

In [ ]:
# Attivare la conversione automatica tra pandas e R
pandas2ri.activate()

# Caricare i pacchetti necessari in R
lme4 = importr('lme4')
stats = importr('stats')

# Convertire le colonne categoriali in stringhe (altrimenti errore)
data_long['Phase'] = data_long['Phase'].astype(str)
data_long['Condition'] = data_long['Condition'].astype(str)
data_long['Training'] = data_long['Training'].astype(str)

# Convertire il DataFrame pandas in un DataFrame R
data_long_r = pandas2ri.py2rpy(data_long)

# Definire il DataFrame in R
ro.globalenv['data_long'] = data_long_r


In [ ]:
# Cambiare i formati delle variabili
ro.r('''data_long$Subject_ID <- as.factor(data_long$Subject_ID)
        data_long$Phase <- as.factor(data_long$Phase)
        data_long$Condition <- as.factor(data_long$Condition)
        data_long$Training <- as.factor(data_long$Training)
''')

#Verificare l'avvenuta conversione
ro.r('''sapply(data_long, class)
''')

In [ ]:
# Eseguire il codice R per il modello GLMM
ro.r('''
library(lme4)

# Modificare la variabile 'Accuracy' per essere binaria
data_long$Accuracy <- as.factor(data_long$Accuracy)

# Specificare il modello GLMM
model <- glmer(Accuracy ~ Condition * Training * Phase + (1 | Subject_ID), 
               data = data_long, 
               family = binomial,
               control = glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 100000)))
               
# Ottenere il sommario del modello
model_summary <- summary(model)
''')

In [ ]:
# Ottenere l'oggetto R model_summary in Python
model_summary = ro.r('model_summary')

# Stampare il sommario
print(model_summary)

In [ ]:
#Eseguire Contrasti con correzione di Bonferroni
ro.r('''

library(emmeans)

# Calcolare le "estimated marginal means"
emm <- emmeans(model, ~ Condition * Training * Phase)

# Visualizzare le emmeans
emm

# Fare Contrasti con conrrezione Bonferroni
pairwise_comparisons <- contrast(emm, method = "pairwise")
results <- summary(pairwise_comparisons, adjust = "bonferroni")

# Filtrare i risultati con p-value < 0.05
significant_results <- results[results$p.value < 0.05, ]

# Visualizzare i risultati significativi
print(significant_results)

''')